In [1]:
# setup
from pyswmm import Simulation
import os
import dask
import time

In [2]:
# nsims
nsims = 2

In [3]:
# specify location
print(os.path.abspath(os.curdir))
os.chdir("..")
os.chdir("probabilistic_python")
dir_path = os.path.abspath(os.curdir)
print(dir_path)

input_path = dir_path + r'\input\swmm\input_'
print(input_path)

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\jupyter_notebooks
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_


## first, without dask

In [4]:
def do_job(i):
    sim_folder = input_path + str(i)
    sim_file = os.path.join(sim_folder, r'JS_NPlesantCreek.inp')
    print(sim_file)

    binary_path = sim_folder + r'\JS_NPlesantCreek.out'

    # delete pre-existing .out in order to run swmm
    if os.path.exists(binary_path):
        os.remove(binary_path)
    else:
        print("Can not delete the file as it doesn't exist")

    # load the model - no interaction, write out binary file
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_path)
    return(sim)
    #sim.execute()

In [5]:
%%time
do_job(1).execute()

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp

... SWMM Version 5.3.0.dev0
... Run CompleteWall time: 57 s


In [15]:
%%time
do_job(2).execute()

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_2\JS_NPlesantCreek.inp
Can not delete the file as it doesn't exist

... SWMM Version 5.3.0.dev0
... Run CompleteWall time: 2min 26s


## Now, with dask

In [4]:
@dask.delayed
def delay_job(i):
    sim_folder = input_path + str(i)
    sim_file = os.path.join(sim_folder, r'JS_NPlesantCreek.inp')
    print(sim_file)

    binary_path = sim_folder + r'\JS_NPlesantCreek.out'

    # delete pre-existing .out in order to run swmm
    if os.path.exists(binary_path):
        os.remove(binary_path)
    else:
        print("Can not delete the file as it doesn't exist")

    # load the model - no interaction, write out binary file
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_path)
    #return(sim)
    sim.execute()
    return("file " + str(i) + " simulated!")

In [8]:
# can it do just 1?
delayed_job = [delay_job(a) for a in range(1,2)]
delayed_job

[Delayed('delay_job-71cde968-ccce-48c6-8ec7-644cc21b466c')]

In [9]:
%%time
dask.delayed(print)(delayed_job).compute()

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp
Can not delete the file as it doesn't exist

... SWMM Version 5.3.0.dev0
... Run Complete['file 1 simulated!']
Wall time: 1min 3s


It appears that the dask package has been applied successfully, and it works perfectly on 1 simulation.

In [10]:
# can it do multiple?
delayed_jobs = [delay_job(a) for a in range(1,nsims+1)]
delayed_jobs

[Delayed('delay_job-9a652c59-2393-414b-bd10-48ad1b3e5b98'),
 Delayed('delay_job-0748fda5-5cd8-4a85-b84a-7e4c50769b4b')]

In [11]:
%%time
dask.delayed(print)(delayed_jobs).compute()

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_2\JS_NPlesantCreek.inp
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp
Can not delete the file as it doesn't exist


SWMMException: 
  ERROR 401: general system error.

Unfortunately, it looks like, even though python and dask can handle simultaneously running two parallel processes, pyswmm cannot. I don't know if that is a problem that can be fixed with multiple computers, a different package, or at all. 

## I'm doubtful, but let's just see if delaying the start of runs on pyswmm does anything.

In [4]:
@dask.delayed
def staggered_delay_job(i):
    sim_folder = input_path + str(i)
    sim_file = os.path.join(sim_folder, r'JS_NPlesantCreek.inp')
    print(sim_file)

    binary_path = sim_folder + r'\JS_NPlesantCreek.out'

    # delete pre-existing .out in order to run swmm
    if os.path.exists(binary_path):
        os.remove(binary_path)
    else:
        print("Can not delete the file as it doesn't exist")
    
    # stagger starting times 2 sec apart
    time.sleep(2*i)
    # load the model - no interaction, write out binary file
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_path)
    #return(sim)
    sim.execute()
    return("file " + str(i) + " simulated!")

In [5]:
# can it do just 1?
staggered_delayed_job = [staggered_delay_job(a) for a in range(1,2)]
staggered_delayed_job

[Delayed('staggered_delay_job-36d0c8b3-98fb-4df0-9579-46853e02c672')]

In [6]:
%%time
dask.delayed(print)(staggered_delayed_job).compute()

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp
Can not delete the file as it doesn't exist

... SWMM Version 5.3.0.dev0
... Run Complete['file 1 simulated!']
Wall time: 57.8 s


Again, it works perfectly on 1 simulation.

In [7]:
# can it do multiple?
staggered_delayed_jobs = [staggered_delay_job(a) for a in range(1,nsims+1)]
staggered_delayed_jobs

[Delayed('staggered_delay_job-d063c475-1f9c-4380-b451-b01d36a11dea'),
 Delayed('staggered_delay_job-dfd064b9-782b-4608-b4a5-a44101834fa7')]

In [ ]:
%%time
dask.delayed(print)(staggered_delayed_jobs).compute()

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp
C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_2\JS_NPlesantCreek.inp
Can not delete the file as it doesn't exist

... SWMM Version 5.3.0.dev0
... SWMM Version 5.3.0.dev0

Yup, now we've done it. We killed the kernel. So it's now official. pyswmm can't handle simultaneous runs. 